# Programming 2C using Python

In [1]:
# Paraemter values for the 1D keyboad are stored in global variables; you can change them
pr_hit = 0.9
pr_miss = 0.1
deg_kb = 2

pr_repeat = 0.3
pr_moveOn = 0.7
deg_sp = 2

# 1. Key functions implemented so far

## getPrTableForPossibleInitialStates(lengthOfWord)

In [2]:
# C++: void getPrTableForPossibleInitialStates(lengthOfWord):
# ==> 
# Python: getPrTableForPossibleInitialStates(prTable, lengthOfWord):
#           return the information in the prTable directly
def getPrTableForPossibleInitialStates(lengthOfWord):
    missDistance = range( 1, lengthOfWord+1 )
    exponentialDegrade = [ (1/deg_sp)**i    for i in missDistance]
    scalingConstant = 1 / sum(exponentialDegrade)
    return [scalingConstant*degrade for degrade in exponentialDegrade]
    

# Test the function to get the probabilities of the possible first states
#      for a word of 3 characters (such as "his" in our handout)


## getPrTableForPossibleInitialStatesGivenTheWord(Word)

In [3]:
# A variant that accomplishes the same thing given a word (as a string)
def getPrTableForPossibleInitialStatesGivenTheWord(Word):
    missDistance = range( 1, len(Word)+1 )
    exponentialDegrade = [ (1/deg_sp)**i    for i in missDistance]
    scalingConstant = 1 / sum(exponentialDegrade)
    return [scalingConstant*degrade for degrade in exponentialDegrade]
    
    
# Test the function to get the probabilities of the possible first states
#      for the word "his" 


## getPrTableForPossibleNextStates(lengthOfWord_Plus1, currentState)

In [4]:
# C++ 
# void getPrTableForPossibleNextStates(double transitionPrTable[], 
#                                      int sizeOfTable, int currentState)
# ==>
# Python 
# getPrTableForPossibleNextStates(lengthOfWord_Plus1, currentState)
# return the transitionPrTable

def getPrTableForPossibleNextStates(lengthOfWord_Plus1, currentState):
    statesAsIndices = range( lengthOfWord_Plus1 )
    distances = [state - currentState for state in statesAsIndices ]
    exponentialDegrade = [ (1/deg_sp)**i if i>0 else 0 for i in distances]
    scalingConstant = pr_moveOn/sum(exponentialDegrade)
    probabilitiesOfPossibleFirstStates = (
        [ scalingConstant*degrade for degrade in exponentialDegrade] )
    probabilitiesOfPossibleFirstStates[currentState] = pr_repeat
    return probabilitiesOfPossibleFirstStates


#Test the implementation
getPrTableForPossibleNextStates(len("his")+1, 0) 


[0.3, 0.39999999999999997, 0.19999999999999998, 0.09999999999999999]

## getPrTableForPossibleNextStatesGivenWord(word, currentState)

In [5]:
# A convenient variant
# Python 
# getPrTableForPossibleNextStates(word, int currentState)
# return the transitionPrTable

def getPrTableForPossibleNextStatesGivenWord(word, currentState):
    lengthOfWord_Plus1 =  len(word) +1
    statesAsIndices = range( lengthOfWord_Plus1 )
    distances = [state - currentState for state in statesAsIndices ]
    exponentialDegrade = [ (1/deg_sp)**i if i>0 else 0 for i in distances]
    scalingConstant = pr_moveOn/sum(exponentialDegrade)
    probabilitiesOfPossibleFirstStates = (
        [ scalingConstant*degrade for degrade in exponentialDegrade] )
    probabilitiesOfPossibleFirstStates[currentState] = pr_repeat
    return probabilitiesOfPossibleFirstStates

#Test the implementation
currentState = 0
getPrTableForPossibleNextStatesGivenWord("his", currentState)

[0.3, 0.39999999999999997, 0.19999999999999998, 0.09999999999999999]

## prCharGiveCharState(x, y)

In [6]:
#Probability of touching x when trying to type y
def prCharGiveCharState(x, y):
    if x==y:
        return pr_hit
    
    diffASCII = range(1,26)
    missdist = [min(n, 26-n) for n in diffASCII ]
    exponentialDegrade = [(1/deg_kb)**i for i in missdist]
    constant_x= pr_miss/sum(exponentialDegrade)
    
    distASCII_x_y = abs( ord(x) - ord(y) ) 
    distKB_x_y = min(distASCII_x_y, 26-distASCII_x_y )
    return constant_x* (1/deg_kb)** distKB_x_y 

## take1SampleFrom1PrSpace

In [7]:
# C++: int take1SampleFrom1PrSpace(double prTable[], int sizeOfTable)
# ==>
# Python: take1SampleFrom1PrSpace(prTable)
#  the size of the table can be implicitly determined 
def take1SampleFrom1PrSpace(prTable):
    probabilityThresholds = np.add.accumulate(prTable)
    sample = np.random.random()
    choice = (sample > probabilityThresholds).sum()
    # print("Sample=", sample, ",\t choice=", choice)
    return choice


import numpy as np
prTable = np.array([0.25, 0.5, 0.25])
[take1SampleFrom1PrSpace(prTable) for i in range(20)]

[1, 1, 2, 2, 1, 1, 1, 1, 1, 0, 1, 0, 2, 0, 1, 2, 1, 1, 1, 2]

In [8]:
#Use bin count to check empirical frequencies observed
np.bincount( [take1SampleFrom1PrSpace(prTable) for i in range(1000000)] )/1000000


array([0.24969 , 0.500366, 0.249944])

## getKeyboardProbabilityTable
### note: this is a simple variant of prCharGiveCharState(x, y)

In [9]:
#C++: void getKeyboardProbabilityTable(char charToType, double prTable[])
#==>
#Python: getKeyboardProbabilityTable(charToType) 
#        to return the probabilities of getting a, b, ..., y, z
#        as a numpy array
#Note: This is simply a simple variant of prCharGiveCharState(x, y):

def getKeyboardProbabilityTable(charToType):
    #First determine the scaling constant for the exponential degrading
    diffASCII = range(1,26)
    missdist = [min(n, 26-n) for n in diffASCII ]
    exponentialDegrade = [(1/deg_kb)**i for i in missdist]
    scalingConstant = pr_miss/sum(exponentialDegrade)    
    
    #Set up an empty probability table 
    prTable = np.empty(26)
    y = charToType
    
    # for each x in a to z,
    # set up a loop to determine the probability of touching x 
    #     when trying to type y (i.e.charToType)
    # store the results in the probability table accordingly
    for i, x in enumerate("abcdefghijklmnopqrstuvwxyz"):
        if x==y:
            prTable[i] = pr_hit
        else:
            distASCII_x_y = abs( ord(x) - ord(y) ) 
            distKB_x_y = min(distASCII_x_y, 26-distASCII_x_y )
            prTable[i] = scalingConstant * (1/deg_kb)** distKB_x_y 
    
    return prTable


#Test the implementation


In [10]:
#More test on the implementation
getKeyboardProbabilityTable('b')

array([2.50045785e-02, 9.00000000e-01, 2.50045785e-02, 1.25022892e-02,
       6.25114462e-03, 3.12557231e-03, 1.56278615e-03, 7.81393077e-04,
       3.90696539e-04, 1.95348269e-04, 9.76741347e-05, 4.88370673e-05,
       2.44185337e-05, 1.22092668e-05, 6.10463342e-06, 1.22092668e-05,
       2.44185337e-05, 4.88370673e-05, 9.76741347e-05, 1.95348269e-04,
       3.90696539e-04, 7.81393077e-04, 1.56278615e-03, 3.12557231e-03,
       6.25114462e-03, 1.25022892e-02])

In [11]:
#More test on the implementation
getKeyboardProbabilityTable('b').sum()

1.0000000000000002

## typeOneChar

In [12]:
# C++: char typeOneChar(char charToType)
# Python: typeOneChar(charToType) 
#   use  take1SampleFrom1PrSpace and
#        getKeyboardProbabilityTable to
#   simulate typing charToType and return the resulting character pressed 

def typeOneChar(charToType):
    keys = "abcdefghijklmnopqrstuvwxyz" 
    prTable = getKeyboardProbabilityTable(charToType)
    indexOfKeyPressed = take1SampleFrom1PrSpace( prTable )
    return keys[ indexOfKeyPressed ]

#Test the implementatiton

#Type 'a' for 10 times and see the results
[typeOneChar('a') for i in range(10) ]

['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']

In [13]:
#More test the implementatiton

#Type 'a' for 10 times under a different setting and see the results
[typeOneChar('a') for i in range(10) ]

['a', 'a', 'x', 'a', 'a', 'a', 'a', 'a', 'a', 'b']

##  typeOneWord

In [14]:
# C++: void typeOneWord( char word[], char output[], 
#                        bool traceON = false, int maxOutput=100)
# Python: typeOneWord( word, trace=False )
#      simulate the typing of the given word (a string) and
#      return the resulted string

def typeOneWord(word, trace=False):
    #Special States
    I_stateIndex = -1 
    F_stateIndex = len(word)
    
    # Step 0: Simulation of leaving the starting state I to enter some (first) state 
    #        to enter a regular state as the current state: throw a dice
    charOutputsObservedSofar = "_"
    stateTrajectorySofar = "I"
    prTable = getPrTableForPossibleInitialStatesGivenTheWord(word)

    currentState_index = take1SampleFrom1PrSpace(prTable)
    currentState_char = word[ currentState_index  ]

    if trace:
        print( "First state reached after leaving I: (index, char)=", 
               (currentState_index, currentState_char) 
             )
    
    while( currentState_index != F_stateIndex): # not the Final state F yet.
        # Step 1: Simulation of typing a character given the current state: throw a dice
        charTyped = typeOneChar(currentState_char)
        charOutputsObservedSofar += charTyped
        stateTrajectorySofar += currentState_char

        if trace:
            print( "Current state: (index, char)=", (currentState_index, currentState_char) )
            print( charTyped, " is pressed when trying to type ", currentState_char)
            print( "char outputs so far:\t", charOutputsObservedSofar )
            print( "state trajectory so far:", stateTrajectorySofar )
            print()

        # Step 2: Simulation of leaving the current state to enter one of the possble next states
        #       : throw a dice
        prTable = getPrTableForPossibleNextStatesGivenWord(word, currentState_index)
        nextState_index = take1SampleFrom1PrSpace(prTable)
        nextState_char = word[ nextState_index  ] if (nextState_index<F_stateIndex) else "F"
        if trace:
            print( "next state to enter: ", (nextState_index, 
                                             nextState_char) )

        currentState_index = nextState_index
        currentState_char = nextState_char 

    if trace:
        print("Finish typing the word ", word)
        print( "char outputs:\t\t", charOutputsObservedSofar+"_" )
        print( "state trajectory:\t", stateTrajectorySofar+"F" )
        
    return charOutputsObservedSofar[1:]

      

In [15]:
pr_hit = 0.9
pr_miss = 0.1
deg_kb = 4

pr_repeat = 0.1
pr_moveOn = 0.9
deg_sp = 4

word = "his"

#See the trace of the simulation of typing one word
typeOneWord( word, True)


First state reached after leaving I: (index, char)= (0, 'h')
Current state: (index, char)= (0, 'h')
h  is pressed when trying to type  h
char outputs so far:	 _h
state trajectory so far: Ih

next state to enter:  (1, 'i')
Current state: (index, char)= (1, 'i')
i  is pressed when trying to type  i
char outputs so far:	 _hi
state trajectory so far: Ihi

next state to enter:  (2, 's')
Current state: (index, char)= (2, 's')
s  is pressed when trying to type  s
char outputs so far:	 _his
state trajectory so far: Ihis

next state to enter:  (3, 'F')
Finish typing the word  his
char outputs:		 _his_
state trajectory:	 IhisF


'his'

In [16]:
# See the result of typing the same word 10000 times
[typeOneWord(word, False) for i in range(10000) ]

['his',
 'his',
 'gis',
 'his',
 'hs',
 'h',
 'is',
 'his',
 'his',
 'hiss',
 'hs',
 'his',
 'hh',
 'jsss',
 'hr',
 'hiii',
 'g',
 'his',
 'hiis',
 'hj',
 'irs',
 'hiss',
 'his',
 'hit',
 'hi',
 'hs',
 'is',
 'i',
 'his',
 'gs',
 't',
 'his',
 'fs',
 'hir',
 'hs',
 'hiss',
 'h',
 'hiis',
 'hiis',
 'iss',
 'his',
 'ii',
 'hks',
 'hit',
 'hh',
 'hhhhis',
 'hs',
 'is',
 'hs',
 'hss',
 'hsss',
 'i',
 'his',
 'his',
 'hgjs',
 'hiss',
 'hs',
 'his',
 's',
 'hiis',
 'his',
 'his',
 'is',
 'his',
 'his',
 'hi',
 'iiit',
 'his',
 'hs',
 'is',
 'his',
 'his',
 'hs',
 'hs',
 'hiss',
 'his',
 's',
 'iis',
 'hi',
 'iis',
 'hhi',
 's',
 'hiis',
 'is',
 'gi',
 'hks',
 'gis',
 'hhs',
 'his',
 'his',
 'gi',
 's',
 'hir',
 'hks',
 'hhis',
 's',
 'ii',
 'his',
 'iit',
 'h',
 'hk',
 'h',
 'is',
 'is',
 'his',
 'ciis',
 'hhs',
 'hkis',
 'hs',
 's',
 'his',
 'hi',
 'r',
 'iss',
 'hhis',
 'is',
 'gs',
 'his',
 'his',
 'his',
 's',
 'his',
 'is',
 'hi',
 'is',
 's',
 'hi',
 'hhhs',
 'hgs',
 'his',
 'his',
 'i

# 2. Basics of file input/output in Python

### 2.1 Basics of file output in Python

In [17]:
# write data into a file. 
Lines = ["Line 1: a;","Line 2: b;","Line 3: c;"]

file = open("outputFile1","w") 
for L in Lines:
    file.write(L) 
file.close()

In [18]:
# %load outputFile1

In [19]:
# write data into a file. 
Lines = ["Line 1: a;","Line 2: b;","Line 3: c;"]

file = open("outputFile2","w") 
file.writelines(Lines) 
file.close()


In [20]:
# %load outputFile2

In [21]:
# write data into a file. 
Lines = ["Line 1: a;","Line 2: b;","Line 3: c;"]

file = open("outputFile3","w") 
for line in Lines:
    file.writelines(line +"\n") 
file.close()


In [22]:
# %load outputFile3



### 2.2 Basics of file intput in Python

In [23]:
# read data from a file. 
file = open("outputFile3","r") 
x=file.read()
print("x is", x)
file.close()


x is Line 1: a;
Line 2: b;
Line 3: c;



In [24]:
file = open("outputFile3","r") 
x=file.readline()
y=file.readline()
z=file.readline()
#show what we got in x, y, z
x, y, z


('Line 1: a;\n', 'Line 2: b;\n', 'Line 3: c;\n')

In [25]:
file = open("outputFile3","r") 
lines=file.readlines()
print("lines is", lines)


lines is ['Line 1: a;\n', 'Line 2: b;\n', 'Line 3: c;\n']


In [26]:
# Strip the white spaces(i.e. spaces, tabs, and specifically'\n' in this case)
strippedLines = [line.strip( ) for line in lines]
strippedLines


['Line 1: a;', 'Line 2: b;', 'Line 3: c;']

In [27]:
# Read words from biolaVision.txt
file = open("biolaVision.txt","r") 
lines = file.readlines()
file.close()

words = [line.strip( ) for line in lines]

print(len(words), " words in biolaVision.txt:\n")
words

72  words in biolaVision.txt:



['biola',
 'university',
 'vision',
 'is',
 'to',
 'be',
 'an',
 'exemplary',
 'christian',
 'university',
 'characterized',
 'as',
 'a',
 'community',
 'of',
 'grace',
 'that',
 'promotes',
 'and',
 'inspires',
 'personal',
 'life',
 'transformation',
 'in',
 'christ',
 'which',
 'illuminates',
 'the',
 'world',
 'with',
 'his',
 'light',
 'and',
 'truth',
 'further',
 'as',
 'a',
 'global',
 'center',
 'for',
 'christian',
 'thought',
 'and',
 'an',
 'influential',
 'evangelical',
 'voice',
 'that',
 'addresses',
 'crucial',
 'cultural',
 'issues',
 'biola',
 'university',
 'aspires',
 'to',
 'lead',
 'with',
 'confidence',
 'and',
 'compassion',
 'an',
 'intellectual',
 'and',
 'spiritual',
 'renewal',
 'that',
 'advances',
 'the',
 'purpose',
 'of',
 'christ']

# 3. Implement typeOneArticle

In [28]:
# C++: void typeOneArticle (const char * corruptedMessageFile, const char * sourceArticle, 
#                           bool trace = false);
# Or
# C++: void typeOneArticle (const string corruptedMessageFile, const string sourceArticle, 
#                           bool trace = false);
# ==>
# Python: typeOneArticle (corruptedMessageFile, sourceArticle, 
#                           trace = False)


In [29]:
def typeOneArticle(corruptedMessageFile, sourceArticle, trace = False):
    file = open(sourceArticle,"r") 
    lines = file.readlines()
    file.close()
    words = [line.strip( ) for line in lines]
    
    corruptedWords = [ typeOneWord(word) for word in words]
    if trace:
        print( corruptedWords )
    
    file = open(corruptedMessageFile,"w") 
    for corruptedWord in corruptedWords:
        file.writelines(corruptedWord+"\n")
    file.close()
    
                        
    

In [30]:
pr_hit = 0.6
pr_miss = 0.4
deg_kb = 2

pr_repeat = 0.2
pr_moveOn = 0.8
deg_sp = 2

typeOneArticle("corruptedVision", "biolaVision.txt")


In [31]:
# %load corruptedVision

# Program 3A

In [32]:
wordToType = "his"
observedString = "he"

## State representations (excluding the special state I and F)

In [33]:
state_indicices = list( range(len(wordToType) ))
state_indicices 

[0, 1, 2]

In [34]:
state_chars = wordToType
state_chars

'his'

In [35]:
states = [ (state_index, state_char) for state_index, state_char in zip(  state_indicices, state_chars)]
states

[(0, 'h'), (1, 'i'), (2, 's')]

## HMM representation (excluding the special state I)

In [36]:
#At the beginning, the initial probability vector Pi after leaving the special state I

vector_pi_list = getPrTableForPossibleInitialStatesGivenTheWord(wordToType)
vector_pi_list

[0.5714285714285714, 0.2857142857142857, 0.14285714285714285]

In [37]:
# Transition probability matrix A (excluding the rows and columns for I and F)
lenthOfWord = len(wordToType)
matrix_A_List = [ getPrTableForPossibleNextStatesGivenWord(wordToType, currentState) for currentState in range(lenthOfWord)]
matrix_A_List 

[[0.2, 0.4571428571428572, 0.2285714285714286, 0.1142857142857143],
 [0.0, 0.2, 0.5333333333333333, 0.26666666666666666],
 [0.0, 0.0, 0.2, 0.8]]

In [38]:
# Access the transition probability $a_{ij}$
i = 0; j =1
matrix_A_List[i][j]

0.4571428571428572

In [39]:
# observation probability matrix B, 
# excluding the rows for I and F columns, 
# exclucing the columns for ReadyToType and EndOfWord
alphabet = "abcdefghijklmnopqrstuvwxyz"
matrix_B_List = [ [prCharGiveCharState(char, state_Char) for char in alphabet] 
                  for state_Char in state_chars]
matrix_B_List 

[[0.001562786154691411,
  0.003125572309382822,
  0.006251144618765644,
  0.012502289237531288,
  0.025004578475062576,
  0.05000915695012515,
  0.1000183139002503,
  0.6,
  0.1000183139002503,
  0.05000915695012515,
  0.025004578475062576,
  0.012502289237531288,
  0.006251144618765644,
  0.003125572309382822,
  0.001562786154691411,
  0.0007813930773457055,
  0.00039069653867285274,
  0.00019534826933642637,
  9.767413466821319e-05,
  4.883706733410659e-05,
  2.4418533667053296e-05,
  4.883706733410659e-05,
  9.767413466821319e-05,
  0.00019534826933642637,
  0.00039069653867285274,
  0.0007813930773457055],
 [0.0007813930773457055,
  0.001562786154691411,
  0.003125572309382822,
  0.006251144618765644,
  0.012502289237531288,
  0.025004578475062576,
  0.05000915695012515,
  0.1000183139002503,
  0.6,
  0.1000183139002503,
  0.05000915695012515,
  0.025004578475062576,
  0.012502289237531288,
  0.006251144618765644,
  0.003125572309382822,
  0.001562786154691411,
  0.0007813930773457

In [40]:
# Access the observation probability $b_{ij}$
i = 0; j =0   #Typing the first character (state_index is 0) in the word but get 'a' generated  
matrix_B_List[i][j]

0.001562786154691411

In [41]:
# Access the observation probability $b_{ij}$
i = 0; j =1   #Typing the first character (state_index is 0) in the word but get 'b' generated   
matrix_B_List[i][j]

0.003125572309382822

## Use np arrays as data structures instead

In [42]:
vector_pi = np.array( vector_pi_list )
matrix_A = np.array( matrix_A_List )
matrix_B = np.array( matrix_B_List )

In [43]:
print(vector_pi)  #Check the shape
print(vector_pi.shape)  #Check the shape
vector_pi.sum()         #Check the sum of probabilties

[0.57142857 0.28571429 0.14285714]
(3,)


1.0

In [44]:
print(matrix_A.shape)  #Check the shape
matrix_A.sum(axis = 1) #Check the sum of probabilties on each row

(3, 4)


array([1., 1., 1.])

In [45]:
print(matrix_B.shape)  #Check the shape
matrix_B.sum(axis = 1) #Check the sum of probabilties on each row

(3, 26)


array([1., 1., 1.])

# Example: You can use lists above as the data structures to implement the work or You may consider using numpy arrays as the data structures to implement the work. Below shows the idea of using Numpy arrays for the implementation of the forward algorithm.

In [46]:
wordToType = "his"
observedString = "ab"


## Stage 1 (The column for observing ReadyToType ): Must start at state I (probability 0 for the other states)

## Stage 2 (The column for observing 'a'): only keep the real states

In [47]:
#Probabilties of transitioning into each of the real states from I
print( vector_pi.shape)
vector_pi

(3,)


array([0.57142857, 0.28571429, 0.14285714])

In [48]:
#Probabilties of transitioning into each of the real states from I
stage2_1 = vector_pi

In [49]:
#Probabilities of observing the first character in each of the real states (excluding F)
observationIndex = 0
charObserved = observedString[observationIndex]
print( "charObserved:", charObserved)
indexOfObservedChar = ord(charObserved) - ord('a')
matrix_B[:, indexOfObservedChar]

charObserved: a


array([0.00156279, 0.00078139, 0.00078139])

In [50]:
# Combine the two items above to calculate
# Probabilities of ending in each of the real states (excluding F) and observing a
stage2_2 = stage2_1*matrix_B[:, indexOfObservedChar]
stage2_2

array([0.00089302, 0.00022326, 0.00011163])

## Stage 3 (The column for observing 'b'): only keep the real states

In [51]:
stage2_2

array([0.00089302, 0.00022326, 0.00011163])

In [52]:
stage2_2[:, np.newaxis]
print( stage2_2[:, np.newaxis].shape )
stage2_2[:, np.newaxis]

(3, 1)


array([[0.00089302],
       [0.00022326],
       [0.00011163]])

In [53]:
#Probabilties of further transitioning into each of the real states
stage3_1 = stage2_2[:, np.newaxis]  * matrix_A
stage3_1 

array([[1.78604132e-04, 4.08238016e-04, 2.04119008e-04, 1.02059504e-04],
       [0.00000000e+00, 4.46510330e-05, 1.19069421e-04, 5.95347107e-05],
       [0.00000000e+00, 0.00000000e+00, 2.23255165e-05, 8.93020660e-05]])

In [54]:
stage3_2 = stage3_1.sum(axis = 0)
stage3_2

array([0.0001786 , 0.00045289, 0.00034551, 0.0002509 ])

In [55]:
#Probabilities of ending in each of the real states (excluding F)
stage3_2[:-1]

array([0.0001786 , 0.00045289, 0.00034551])

In [56]:
#Probabilities of observing the second character in each of the real states (excluding F)
observationIndex = 1
charObserved = observedString[observationIndex]
print( "charObserved:", charObserved)
indexOfObservedChar = ord(charObserved) - ord('a')
matrix_B[:, indexOfObservedChar]


charObserved: b


array([0.00312557, 0.00156279, 0.0003907 ])

In [57]:
# Combine the two items above to calculate
# Probabilities of ending in each of the real states (excluding F) and observing b
stage3_3 = stage3_2[:-1]*matrix_B[:, indexOfObservedChar]
stage3_3

array([5.58240129e-07, 7.07768735e-07, 1.34991103e-07])

## Stage 4 (The column for observing EndOfWord ): Must be in the Final State F (probability = 0 for the other states)

In [58]:
# Probabilities of ending in each of the real states (excluding F) and observing b
stage3_3

array([5.58240129e-07, 7.07768735e-07, 1.34991103e-07])

In [59]:
# Probabilities of then transitioning to F from each of the real states
matrix_A[:, len( wordToType)]

array([0.11428571, 0.26666667, 0.8       ])

In [60]:
# Combine the two items above to calculate the
# Probabilities of then transition to F and ending the typing process of the word
# Transisiton to F (index 3 in this case )
stage4_1 = stage3_3 * matrix_A[:, len( wordToType)]
stage4_1

array([6.37988719e-08, 1.88738329e-07, 1.07992882e-07])

In [61]:
# The result is the sum of all these probabilities

In [62]:
result = stage4_1.sum()
result

3.6053008344833416e-07

## The result above is also what you would get from the sample demo program for HMMs.
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

# Program 3A_Complete ==> 
## 1. Define a function prOf1CharSeriesWhenTyping1Word_F to generalize the process (of the forward algorithm) demonstrated in the case above for any given wordToType and any given observedString

In [63]:
# Using the forward algorithm algorithm to determine the probability: 
# The function should calculate and return
#     the probability of getting the string d 
#     when the user (modelled by the parameter values of pr_hit, pr_repeat, degenerate_kb, ...)
#     want to type the word in string w
# When the trace is True, the function will report the trace of computation done.

def prOf1CharSeriesWhenTyping1Word_F(observedString, wordToType, trace = False):
    #The probability distribution after leaving I
    vector_pi_list = getPrTableForPossibleInitialStatesGivenTheWord(wordToType)
    
    #The transition probability matrix A
    lenthOfWord = len(wordToType)
    matrix_A_List = [ getPrTableForPossibleNextStatesGivenWord(wordToType, currentState) 
                      for currentState in range(lenthOfWord)]
    
    #The observation probability matrix B
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    matrix_B_List = [ [prCharGiveCharState(char, state_Char) for char in alphabet] 
                      for state_Char in wordToType]
    
    ##############################################
    #Cast them into numpy arrays
    ##############################################
    vector_pi = np.array( vector_pi_list )
    matrix_A = np.array( matrix_A_List )
    matrix_B = np.array( matrix_B_List )
    
    if trace == True:
        print("vector_pi", vector_pi.shape, ":\n", vector_pi)
        print("matrix_A", matrix_A.shape, ":\n", matrix_A)
        print("matrix_B", matrix_B.shape, ":\n", matrix_B)
    
    ##############################################
    #For the first column (corresponding to the first character observed)
    ##############################################
    observationIndex = 0
    charObserved = observedString[observationIndex]
    if trace == True:
        print( "\n\nobservationIndex, charObserved:", observationIndex, ",", charObserved)
    indexOfObservedCharInAlphabet = ord(charObserved) - ord('a');
    
    #transitionProbabilties record the 1st-stage results of a column regarding 
    #    the probabilities of ending in each of the states at this point
    transitionProbabilties = vector_pi
    if trace == True:
        print("Probabilties of ending at the states at this point:\n", transitionProbabilties)
    
    #columnProbabilities record the 2nd-stage results of a column regarding 
    #    the probabilities of ending in each of the states at this point and also
    #                         seeing the specific character at this point
    observationProbabilities = matrix_B[:, indexOfObservedCharInAlphabet]
    if trace == True:
        print("probabilities of observing ", charObserved, " at specific states alone:\n", 
              observationProbabilities)

    columnProbabilities = transitionProbabilties * observationProbabilities
    if trace == True:
        print("Probabilties of observing up to", charObserved, 
              " and ending at the states at this point:\n", columnProbabilities)
    
    
    ##############################################
    # For the remaining columns one at a time
    ##############################################
    lenthOfObservedString = len(observedString)
    for observationIndex in np.arange(1, lenthOfObservedString):
        charObserved = observedString[observationIndex]
        if trace == True:
            print( "\nobservationIndex, charObserved:", observationIndex, ",", charObserved)
        
        transitionProbabilties = (columnProbabilities[:, np.newaxis] * matrix_A).sum(axis = 0)
        transitionProbabilties = transitionProbabilties[:-1]  # Drop the probability to F
        if trace == True:
            print("Probabilties of ending at the states at this point:\n", transitionProbabilties)

        indexOfObservedCharInAlphabet = ord(charObserved) - ord('a')
        observationProbabilities = matrix_B[:, indexOfObservedCharInAlphabet]
        if trace == True:
            print("probabilities of observing ", charObserved, " at specific states alone:\n", 
                  observationProbabilities)

        columnProbabilities = (transitionProbabilties) * observationProbabilities
        if trace == True:
            print("Probabilties of observing up to", charObserved, 
                  " and ending at the states at this point:\n", columnProbabilities)
    
    ##############################################
    # Determine the sum of probabilities of transitioning to the Final state F from each state
    ##############################################
    if trace == True:
        print("\nprobabilities of transitioning to F from states at this point:\n", 
              matrix_A[:, len( wordToType)] )
    pr = (columnProbabilities * matrix_A[:, len( wordToType)]).sum()
    if trace == True:
        print("\nSum of the probabilitie above:", pr);
    
    return pr 


In [64]:
# With trace
prOf1CharSeriesWhenTyping1Word_F("his", "his", True)

vector_pi (3,) :
 [0.57142857 0.28571429 0.14285714]
matrix_A (3, 4) :
 [[0.2        0.45714286 0.22857143 0.11428571]
 [0.         0.2        0.53333333 0.26666667]
 [0.         0.         0.2        0.8       ]]
matrix_B (3, 26) :
 [[1.56278615e-03 3.12557231e-03 6.25114462e-03 1.25022892e-02
  2.50045785e-02 5.00091570e-02 1.00018314e-01 6.00000000e-01
  1.00018314e-01 5.00091570e-02 2.50045785e-02 1.25022892e-02
  6.25114462e-03 3.12557231e-03 1.56278615e-03 7.81393077e-04
  3.90696539e-04 1.95348269e-04 9.76741347e-05 4.88370673e-05
  2.44185337e-05 4.88370673e-05 9.76741347e-05 1.95348269e-04
  3.90696539e-04 7.81393077e-04]
 [7.81393077e-04 1.56278615e-03 3.12557231e-03 6.25114462e-03
  1.25022892e-02 2.50045785e-02 5.00091570e-02 1.00018314e-01
  6.00000000e-01 1.00018314e-01 5.00091570e-02 2.50045785e-02
  1.25022892e-02 6.25114462e-03 3.12557231e-03 1.56278615e-03
  7.81393077e-04 3.90696539e-04 1.95348269e-04 9.76741347e-05
  4.88370673e-05 2.44185337e-05 4.88370673e-05 9.76

0.02570773798355381

In [65]:
# No trace
prOf1CharSeriesWhenTyping1Word_F("his", "his")

0.02570773798355381

In [66]:
prOf1CharSeriesWhenTyping1Word_F("sadasff", "parameters")

1.180732321622022e-11

## 2. Define a function prOf1CharSeriesWhenTyping1Word_B to implement a brute-force version for any given wordToType and any given observedString

In [67]:
# A function to return the next state trajectory as a 1 dimensional numpy array
def getNextTrajectory(currentTrajectory, sizeOfStateSpace, trace = False):
    if np.any(currentTrajectory < sizeOfStateSpace-1) == False: 
         return np.zeros( currentTrajectory.shape[0] )
    
    incrementPoint = currentTrajectory.shape[0] -1
    while currentTrajectory[incrementPoint] == sizeOfStateSpace-1:
        incrementPoint -= 1
    if trace: print("incrementPoint", incrementPoint )
    nextTrajectory  = currentTrajectory.copy()
    nextTrajectory[ incrementPoint  ] += 1
    nextTrajectory[ incrementPoint+1:  ] = 0
    return nextTrajectory


trajectory = np.array([0,0])
print( trajectory )
for i in np.arange(15):
    trajectory = getNextTrajectory(trajectory, 4)
    print( trajectory )





[0 0]
[0 1]
[0 2]
[0 3]
[1 0]
[1 1]
[1 2]
[1 3]
[2 0]
[2 1]
[2 2]
[2 3]
[3 0]
[3 1]
[3 2]
[3 3]


In [68]:
# A more efficient variant of the function above:
#   transformToNextTrajectory simply updates the contents of current trajectory
#   to the next trajectory
def transformToNextTrajectory_x(currentTrajectory, sizeOfStateSpace, trace = False):
    if np.any(currentTrajectory < sizeOfStateSpace-1) == False: 
         return np.zeros( currentTrajectory.shape[0] )
    
    incrementPoint = currentTrajectory.shape[0] -1
    while currentTrajectory[incrementPoint] == sizeOfStateSpace-1:
        incrementPoint -= 1
    if trace: print("incrementPoint", incrementPoint )
    currentTrajectory[ incrementPoint  ] += 1
    currentTrajectory[ incrementPoint+1:  ] = 0



sizeOfStateSpace = 4
trajectory = np.array([0,0,0])
print( trajectory )
while np.all(trajectory == sizeOfStateSpace-1) == False:
    transformToNextTrajectory_x(trajectory, sizeOfStateSpace)
    print( trajectory )




[0 0 0]
[0 0 1]
[0 0 2]
[0 0 3]
[0 1 0]
[0 1 1]
[0 1 2]
[0 1 3]
[0 2 0]
[0 2 1]
[0 2 2]
[0 2 3]
[0 3 0]
[0 3 1]
[0 3 2]
[0 3 3]
[1 0 0]
[1 0 1]
[1 0 2]
[1 0 3]
[1 1 0]
[1 1 1]
[1 1 2]
[1 1 3]
[1 2 0]
[1 2 1]
[1 2 2]
[1 2 3]
[1 3 0]
[1 3 1]
[1 3 2]
[1 3 3]
[2 0 0]
[2 0 1]
[2 0 2]
[2 0 3]
[2 1 0]
[2 1 1]
[2 1 2]
[2 1 3]
[2 2 0]
[2 2 1]
[2 2 2]
[2 2 3]
[2 3 0]
[2 3 1]
[2 3 2]
[2 3 3]
[3 0 0]
[3 0 1]
[3 0 2]
[3 0 3]
[3 1 0]
[3 1 1]
[3 1 2]
[3 1 3]
[3 2 0]
[3 2 1]
[3 2 2]
[3 2 3]
[3 3 0]
[3 3 1]
[3 3 2]
[3 3 3]


In [69]:
# An even more efficient variant of the function above:
#   only produce trajectories with non-decreasing state indices inside
def transformToNextTrajectory(currentTrajectory, sizeOfStateSpace, trace = False):
    if np.any(currentTrajectory < sizeOfStateSpace-1) == False: 
         return np.zeros( currentTrajectory.shape[0] )
    
    incrementPoint = currentTrajectory.shape[0] -1
    while currentTrajectory[incrementPoint] == sizeOfStateSpace-1:
        incrementPoint -= 1
    if trace: print("incrementPoint", incrementPoint )
    currentTrajectory[ incrementPoint  ] += 1
    currentTrajectory[ incrementPoint+1:  ] =  currentTrajectory[ incrementPoint  ]



sizeOfStateSpace = 4
trajectory = np.array([0,0,0])
print( trajectory )
while np.all(trajectory == sizeOfStateSpace-1) == False:
    transformToNextTrajectory(trajectory, sizeOfStateSpace)
    print( trajectory )




[0 0 0]
[0 0 1]
[0 0 2]
[0 0 3]
[0 1 1]
[0 1 2]
[0 1 3]
[0 2 2]
[0 2 3]
[0 3 3]
[1 1 1]
[1 1 2]
[1 1 3]
[1 2 2]
[1 2 3]
[1 3 3]
[2 2 2]
[2 2 3]
[2 3 3]
[3 3 3]


In [70]:
# Using the brute-force algorithm to determine the probability: 
# The function should calculate and return
#     the probability of getting the string d 
#     when the user (modelled by the parameter values of pr_hit, pr_repeat, degenerate_kb, ...)
#     want to type the word in string w
# When the trace is True, the function will report the trace of computation done.


def prOf1CharSeriesWhenTyping1Word_B(observedString, wordToType, trace = False):
    #The probability distribution after leaving I
    vector_pi_list = getPrTableForPossibleInitialStatesGivenTheWord(wordToType)
    
    #The transition probability matrix A
    lenthOfWord = len(wordToType)
    matrix_A_List = [ getPrTableForPossibleNextStatesGivenWord(wordToType, currentState) 
                      for currentState in range(lenthOfWord)]
    
    #The observation probability matrix B
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    matrix_B_List = [ [prCharGiveCharState(char, state_Char) for char in alphabet] 
                      for state_Char in wordToType]
    
    ##############################################
    #Cast them into numpy arrays
    ##############################################
    vector_pi = np.array( vector_pi_list )
    matrix_A = np.array( matrix_A_List )
    matrix_B = np.array( matrix_B_List )
    
    if trace == True:
        print("vector_pi", vector_pi.shape, ":\n", vector_pi)
        print("matrix_A", matrix_A.shape, ":\n", matrix_A)
        print("matrix_B", matrix_B.shape, ":\n", matrix_B)
    
    #The trajectory should have the same length of observedString.
    #Let's start from [0, ..., 0]
    sizeOfStateSpace = len( wordToType)
    lengthOfTrajectory = len(observedString)
    trajectory = np.zeros( lengthOfTrajectory, dtype="int32" ) 
    pr = 0
    
    allTrajectoriesExamined = False 
    while( allTrajectoriesExamined == False):
        currentStateIndex = trajectory[ 0 ]
        if trace == True:
            print("type(currentStateIndex )", type(currentStateIndex ) )
            print("currentStateIndex=", currentStateIndex )
            print("trajectory[ 0 ]=", trajectory[ 0 ] )
        prTrajectory = vector_pi[ currentStateIndex  ]

        #Check each observation 
        for observationIndex in np.arange(0, len(observedString) ): 
            currentStateIndex = trajectory[ observationIndex ]     
                
            #Multiply the observation probability
            charObserved = observedString[ observationIndex ]
            observedCharIndex =  ord(charObserved) -  ord('a')
            prTrajectory  *=  matrix_B[ currentStateIndex, observedCharIndex]

            #Check whether it is the end of observation
            if observationIndex == len(observedString)-1 : 
                nextStateIndex =  len( wordToType)      #the special final state F as the end
            else:
                nextStateIndex = trajectory[ observationIndex + 1]  # a regular next state

            #Multiply the transition probability to the next state
            prTrajectory  *=  matrix_A[ currentStateIndex, nextStateIndex]

        #Add the probability of this trajectory and observation to the total probability pr
        pr += prTrajectory
        
        transformToNextTrajectory(trajectory, sizeOfStateSpace)
        if np.all(trajectory == (sizeOfStateSpace-1) ):
            allTrajectoriesExamined = True
    

    
    return pr 


In [71]:
prOf1CharSeriesWhenTyping1Word_B("his", "his")

0.025707737931218794

In [72]:
prOf1CharSeriesWhenTyping1Word_B("hsdfsrrerdr", "his")

4.3796480158574565e-27

In [73]:
prOf1CharSeriesWhenTyping1Word_B("paramdguers", "parameters")

3.2333754151142193e-10

In [74]:
#Programming 3B Probabilistic Reasoning for Identity Recognition

def logPrOfGettingDocument1WhenTypingDocument2(document1,document2):

    lines1=[]
    lines2=[]
    pr=0
    logsumEBase=0
    logsumTBase=0
    with open(document1) as f1:
        lines1 = f1.read().splitlines()

    with open(document2) as f2:
        lines2 = f2.read().splitlines()
        
    for i in range (len(lines1)):
        pr=prOf1CharSeriesWhenTyping1Word_F(lines1[i],lines2[i])
        logsumEBase += np.log(pr)
        logsumTBase += np.log10(pr)

    return logsumEBase

In [75]:
#parameters of Johnny
pr_hit = 0.9
pr_miss = 0.1
deg_kb = 2

pr_repeat = 0.1
pr_moveOn = 0.9
deg_sp = 2

logPrOfGettingDocument1WhenTypingDocument2('A.txt','BiolaVision.txt')

-439.5404046807038

In [76]:
logPrOfGettingDocument1WhenTypingDocument2('B.txt','BiolaVision.txt')

-641.4025369739651

In [77]:
logPrOfGettingDocument1WhenTypingDocument2('C.txt','BiolaVision.txt')

-699.6348485807368

In [78]:
logPrOfGettingDocument1WhenTypingDocument2('D.txt','BiolaVision.txt')

-1063.5549122297623

In [79]:
logPrOfGettingDocument1WhenTypingDocument2('E.txt','BiolaVision.txt')

-931.3648640160923

In [80]:
logPrOfGettingDocument1WhenTypingDocument2('F.txt','BiolaVision.txt')

-697.3311239291999

In [81]:
logPrOfGettingDocument1WhenTypingDocument2('G.txt','BiolaVision.txt')

-633.2059107406498

In [82]:
logPrOfGettingDocument1WhenTypingDocument2('H.txt','BiolaVision.txt')

-446.0231356821852

In [83]:
#parameters of Winnie
pr_hit = 0.7
pr_miss = 0.3
deg_kb = 2

pr_repeat = 0.1
pr_moveOn = 0.9
deg_sp = 2

logPrOfGettingDocument1WhenTypingDocument2('A.txt','BiolaVision.txt')

-474.2551121768965

In [84]:
logPrOfGettingDocument1WhenTypingDocument2('B.txt','BiolaVision.txt')

-612.579738507073

In [85]:
logPrOfGettingDocument1WhenTypingDocument2('C.txt','BiolaVision.txt')

-741.9446817984312

In [86]:
logPrOfGettingDocument1WhenTypingDocument2('D.txt','BiolaVision.txt')

-1009.3924249674283

In [87]:
logPrOfGettingDocument1WhenTypingDocument2('E.txt','BiolaVision.txt')

-882.2002539475624

In [88]:
logPrOfGettingDocument1WhenTypingDocument2('F.txt','BiolaVision.txt')

-739.2546513390956

In [89]:
logPrOfGettingDocument1WhenTypingDocument2('G.txt','BiolaVision.txt')

-610.5605843654805

In [90]:
logPrOfGettingDocument1WhenTypingDocument2('H.txt','BiolaVision.txt')

-475.8864122347405

In [91]:
#Parameters of Manny
pr_hit = 0.9
pr_miss = 0.1
deg_kb = 2

pr_repeat = 0.3
pr_moveOn = 0.7
deg_sp = 2

logPrOfGettingDocument1WhenTypingDocument2('A.txt','BiolaVision.txt')

-472.1358651513194

In [92]:
logPrOfGettingDocument1WhenTypingDocument2('B.txt','BiolaVision.txt')

-670.7067379701124

In [93]:
logPrOfGettingDocument1WhenTypingDocument2('C.txt','BiolaVision.txt')

-640.6010651341728

In [94]:
logPrOfGettingDocument1WhenTypingDocument2('D.txt','BiolaVision.txt')

-976.0387872888258

In [95]:
logPrOfGettingDocument1WhenTypingDocument2('E.txt','BiolaVision.txt')

-887.3323815004002

In [96]:
logPrOfGettingDocument1WhenTypingDocument2('F.txt','BiolaVision.txt')

-641.9188791377679

In [97]:
logPrOfGettingDocument1WhenTypingDocument2('G.txt','BiolaVision.txt')

-651.0236324573995

In [98]:
logPrOfGettingDocument1WhenTypingDocument2('H.txt','BiolaVision.txt')

-473.50753787011337

In [99]:
#Parameters of Cathy
pr_hit = 0.7
pr_miss = 0.3
deg_kb = 2

pr_repeat = 0.3
pr_moveOn = 0.7
deg_sp = 2

logPrOfGettingDocument1WhenTypingDocument2('A.txt','BiolaVision.txt')

-507.0782637104098

In [100]:
logPrOfGettingDocument1WhenTypingDocument2('B.txt','BiolaVision.txt')

-643.7515725892246

In [101]:
logPrOfGettingDocument1WhenTypingDocument2('C.txt','BiolaVision.txt')

-685.8225115114949

In [102]:
logPrOfGettingDocument1WhenTypingDocument2('D.txt','BiolaVision.txt')

-924.4600385213306

In [103]:
logPrOfGettingDocument1WhenTypingDocument2('E.txt','BiolaVision.txt')

-841.4990901914263

In [104]:
logPrOfGettingDocument1WhenTypingDocument2('F.txt','BiolaVision.txt')

-686.0554791862593

In [105]:
logPrOfGettingDocument1WhenTypingDocument2('G.txt','BiolaVision.txt')

-631.4261809240516

In [106]:
logPrOfGettingDocument1WhenTypingDocument2('H.txt','BiolaVision.txt')

-504.92036946575115

In [107]:
def learnBestParameterValuesGivenDocument1WhenTypingDocument2(d1,d2):
    global pr_hit,pr_miss,pr_repeat,pr_moveOn
    pr_hit_saved,pr_miss_saved=pr_hit,pr_miss
    pr_repeat_saved,pr_moveOn_saved=pr_repeat,pr_moveOn
   
    pr_hit_list=np.arange(0.1,1.1,0.1)
    pr_repeat_list=np.arange(0.1,1.1,0.1)
   

    higher_prob=np.NINF
   
    for hit in pr_hit_list:
        pr_hit=hit
        pr_miss=1-pr_hit
        for repeat in pr_repeat_list:
            pr_repeat=repeat
            pr_moveOn=1-pr_repeat
           
            p = logPrOfGettingDocument1WhenTypingDocument2(d1,d2)
           
            if p > higher_prob:
                higher_prob=p
                best_pr_hit=pr_hit
                best_pr_repeat=pr_repeat
    pr_hit,pr_miss=pr_hit_saved,pr_miss_saved
    pr_repeat,pr_moveOn=pr_repeat_saved,pr_moveOn_saved
    return [best_pr_hit,1-best_pr_hit,best_pr_repeat,1-best_pr_repeat]

## Program 4B Gradient Ascent

In [132]:
def learnBestParameterValuesGivenDocument1WhenTypingDocument2_GradientAscent (document1, document2,stepSize=0.01):
    global pr_hit,pr_miss,pr_repeat,pr_moveOn
    
    pr_hit_saved,pr_miss_saved=pr_hit,pr_miss
    pr_repeat_saved,pr_moveOn_saved=pr_repeat,pr_moveOn
    
    pr_hit=np.random.rand()*0.6+0.2
    pr_miss=1-pr_hit
    pr_repeat=np.random.rand()*0.6+0.2
    pr_moveOn=1-pr_repeat
    pr=logPrOfGettingDocument1WhenTypingDocument2(document1,document2)
    
    searching=True
    while searching:
        pr_hit_saved2=pr_hit
        pr_hit=pr_hit+stepSize
        pr_miss=1-pr_hit
        pr2=logPrOfGettingDocument1WhenTypingDocument2(document1,document2)
        gradient_hit=pr2-pr
        pr_hit=pr_hit_saved2
        pr_miss=1-pr_hit
        
        pr_repeat_saved2=pr_repeat
        pr_repeat=pr_repeat+stepSize
        pr_moveOn=1-pr_repeat
        pr2=logPrOfGettingDocument1WhenTypingDocument2(document1,document2)
        gradient_repeat=pr2-pr
        pr_repeat=pr_repeat_saved2
        pr_moveOn=1-pr_repeat
        
        rate=stepSize/np.sqrt(gradient_hit**2+gradient_repeat**2)
        pr_hit=pr_hit+gradient_hit*rate
        pr_repeat=pr_repeat+gradient_repeat*rate
        pr_miss=1-pr_hit
        pr_moveOn=1-pr_repeat
        
        pr2=logPrOfGettingDocument1WhenTypingDocument2(document1,document2)
        difference=pr2-pr
        
        if difference >= 0.00000001:
            pr=pr2
            
        if difference < 0.00000001:
            searching=False
            pr_hit=pr_hit_saved2
            pr_repeat=pr_repeat_saved2
            pr_miss=1-pr_hit
            pr_moveOn=1-pr_repeat
            
        
    pr_hit,pr_miss=pr_hit_saved,pr_miss_saved
    pr_repeat,pr_moveOn=pr_repeat_saved,pr_moveOn_saved
    return (pr_hit_saved2,1-pr_hit_saved2,pr_repeat_saved2,1-pr_repeat_saved2)

In [133]:
learnBestParameterValuesGivenDocument1WhenTypingDocument2_GradientAscent('A.txt','BiolaVision.txt')

(0.9170543896210984,
 0.08294561037890158,
 0.08285622539824317,
 0.9171437746017568)

In [134]:
learnBestParameterValuesGivenDocument1WhenTypingDocument2_GradientAscent('B.txt','BiolaVision.txt')

(0.7220360434274706,
 0.27796395657252937,
 0.0771313887125728,
 0.9228686112874271)

In [135]:
learnBestParameterValuesGivenDocument1WhenTypingDocument2_GradientAscent('C.txt','BiolaVision.txt')

(0.9101324108899939,
 0.08986758911000614,
 0.3082043848526873,
 0.6917956151473127)

In [136]:
learnBestParameterValuesGivenDocument1WhenTypingDocument2_GradientAscent('D.txt','BiolaVision.txt')

(0.7017992533586896,
 0.29820074664131035,
 0.36757381373481157,
 0.6324261862651884)

In [137]:
learnBestParameterValuesGivenDocument1WhenTypingDocument2_GradientAscent('E.txt','BiolaVision.txt')

(0.6976843692879686,
 0.3023156307120314,
 0.2907899259410318,
 0.7092100740589682)

In [138]:
learnBestParameterValuesGivenDocument1WhenTypingDocument2_GradientAscent('F.txt','BiolaVision.txt')

(0.9077884757424943,
 0.09221152425750567,
 0.30544506751291167,
 0.6945549324870883)

In [139]:
learnBestParameterValuesGivenDocument1WhenTypingDocument2_GradientAscent('G.txt','BiolaVision.txt')

(0.7383733775908288,
 0.26162662240917123,
 0.12352476307394838,
 0.8764752369260516)

In [140]:
learnBestParameterValuesGivenDocument1WhenTypingDocument2_GradientAscent('H.txt','BiolaVision.txt')

(0.909305796812431,
 0.09069420318756904,
 0.09077879341303285,
 0.9092212065869671)